In [26]:
import sqlite3

In [6]:
import pandas as pd

def load_data(path=None):
    if path is None:
        path = "../data/train_triplets.txt"

    data = pd.read_csv(path, sep="\t", names=["user_id", "song_id", "play_count"])
    print(data.head(5))

    return data

In [7]:
data = load_data()

                                    user_id             song_id  play_count
0  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAKIMP12A8C130995           1
1  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAPDEY12A81C210A9           1
2  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBBMDR12A8C13253B           2
3  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBFNSP12AF72A0E22           1
4  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBFOVM12A58A7D494           1


In [8]:
print(data.shape)

(48373586, 3)


In [21]:
def analyse_play_count_by(data, column_to_group, column_to_sum, ascending=False):
    result = data.groupby(column_to_group)[column_to_sum].sum().reset_index()
    result = result.sort_values(by=column_to_sum, ascending=ascending)
    return result

In [22]:
play_count_by_user = analyse_play_count_by(data, "user_id", "play_count", False)
print(play_count_by_user.head(5))

                                         user_id  play_count
36591   093cb74eb3c517c5179ae24caf0ebec51b24d2a2       13132
69497   119b7c88d58d0c6eb051365c103da5caf817bea6        9884
252820  3fa44653315697f42410a30cb766a4eb102080bb        8210
646483  a2679496cd0af9779a92a13ff7c6af5c81ea8c7b        7015
859158  d7d2d888ae04d16e994d6964214a1de81392ee04        6494


In [23]:
play_count_by_song = analyse_play_count_by(data, "song_id", "play_count", False)
print(play_count_by_song.head(5))

                   song_id  play_count
25043   SOBONKR12A58A7A7E0      726885
12936   SOAUWYT12A81C206F1      648239
287415  SOSXLTC12AF72A7F54      527893
90798   SOFRQTD12A81C233C0      425463
67917   SOEGIYH12A6D4FC0E3      389880


In [25]:
# 取前10W个用户、前3W首歌曲
user_id_list = play_count_by_user.head(100000)
song_id_list = play_count_by_song.head(30000)

print(user_id_list.shape)
print(song_id_list.shape)

(100000, 2)
(30000, 2)


In [31]:
# 读取歌曲详细信息数据
# 处理 track_metadata.db
db_file = "../data/track_metadata.db"
# cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# table_names = cursor.fetchall()
# print("数据库中的表:", table_names)
# 获取表的创建语句
# sql_column = "select sql from sqlite_master where type='table' and name='songs'"
# cursor.execute(sql_column)
# create_statement = cursor.fetchall()[0]
# print(f"表songs的创建语句：{create_statement}")

# head_sql = "select * from songs limit 5"
# cursor.execute(head_sql)
# result = cursor.fetchall()
# print(f"songs表的前5行数据：{result}")

def get_song_info(db_file, table_name, column_names, condition):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    sql = ""
    if 1 == len(column_names):
        sql = f"select {column_names} from {table_name} where condition"
    else:
        sql = f"select {", ".join(column_names)} from {table_name} where condition"
    
    cursor.execute(sql)
    result = cursor.fetchall()

    cursor.close()
    conn.close()

    return result


In [40]:
# 统计歌曲相似度
def calc_song_played_user_count(song_played_id, play_data):
    song_played_user_list = play_data[play_data["song_id"] == song_played_id]["user_id"].to_list()
    # song_played_user_count = len(song_played_user_list)

    return song_played_user_list

def calc_similarity(song_played_id, recommend_list, play_data):
    """
    根据用户的播放记录，统计已播放歌曲，与推荐列表中的歌曲的相似度
    """
    similarity_list = []
    song_played_user_list = calc_song_played_user_count(song_played_id, play_data)

    for song_id in recommend_list:
        song_played_list = calc_song_played_user_count(song_id, play_data)

        common_count = len(set(song_played_user_list) & set(song_played_list))
        total_count = len(set(song_played_user_list) | set(song_played_list))

        similarity_list.append(common_count / total_count)

    return similarity_list, max(similarity_list)

unique_hot_song = play_count_by_song["song_id"].unique()
hot_play_data = data.loc[data["user_id"].isin(user_id_list["user_id"])]
result, most_recomend_song_id = calc_similarity("SOBONKR12A58A7A7E0", unique_hot_song, hot_play_data)

print(result)

KeyboardInterrupt: 